# trajectory_buoy_erddap_forecast

like trajectory_buoy_erddap but instead of nowcast data, uses forecast data, which doesn't have 6 hour gaps wow

https://erddap.sccoos.org/erddap/files/roms_fcst/

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
from parcels import FieldSet
import xarray as xr

import utils
from parcels_utils import buoycsv_to_particleds, clean_erddap_ds, HFRGrid, convert_lon_readings
from parcels_sim import simulation, generate_sim_gif
from plot_utils import plot_trajectories, plot_points_fieldset

In [ ]:
buoy_id = "689-02"
csv_path = utils.WAVEBUOY_DATA_DIR / f"wavebuoy_{buoy_id}.csv"
buoy_ds = buoycsv_to_particleds(csv_path)
buoy_ds

In [ ]:
erddap_paths = [
    utils.CURRENT_NETCDF_DIR / "ca_subCA_fcst_2020072703.nc",
    utils.CURRENT_NETCDF_DIR / "ca_subCA_fcst_2020072803.nc",
    utils.CURRENT_NETCDF_DIR / "ca_subCA_fcst_2020072903.nc",
]
def open_erddap(path):
    dataset = xr.open_dataset(path, drop_variables=["temp", "salt", "zeta"])
    return dataset.sel(depth=0.0)
    
erddap_datasets = list(map(open_erddap, erddap_paths))

In [ ]:
first = erddap_datasets[0]["time"]
for ds in erddap_datasets:
    print((ds["time"] == first).all().values)

In [ ]:
ds_total = None
for ds in erddap_datasets:
    if ds_total is None:
        ds_total = ds
    else:
        ds_total = xr.concat([ds_total, ds], dim="time")
ds_total = ds_total.assign_coords({"lon": convert_lon_readings(ds_total["lon"].values)})

In [ ]:
ds_total = ds_total.sel(lat=slice(33, 35), lon=slice(-122, -120.5))
ds_total

In [ ]:
grid = HFRGrid(ds_total)

In [ ]:
plot_points_fieldset(buoy_ds["lat"].values[0], buoy_ds["lon"].values[0], 0, grid, domain=grid.get_domain(), line=True)

In [ ]:
buoy_ds_sliced = buoy_ds.where((buoy_ds["time"] >= ds_total["time"].min().values) & (buoy_ds["time"] <= ds_total["time"].max().values), drop=True)

In [ ]:
plot_points_fieldset(buoy_ds_sliced["lat"].values[0], buoy_ds_sliced["lon"].values[0], 0, grid, domain=grid.get_domain(), line=True)

In [ ]:
buoy_timestamps = buoy_ds_sliced["time"].values[0].astype("datetime64[s]")
start = buoy_timestamps[0]
end = buoy_timestamps[-1]
start_pos = (buoy_ds_sliced["lat"].isel(traj=0,obs=0).values.item(), buoy_ds_sliced["lon"].isel(traj=0,obs=0).values.item())

In [ ]:
parcels_cfg = {
    "time_range": [start, end],
    "repeat_dt": -1,
    "particles_per_dt": -1,
    "max_variation": 0.0,
    "spawn_points": [
        start_pos
    ],
    "random_spawn": True,
    "simulation_dt": 300,
    "snapshot_interval": 7200,
    "save_snapshots": True,
    "shown_domain": grid.get_domain()
}

In [ ]:
pfile_path, snap_path = simulation("erddap_buoy_forecast", grid, parcels_cfg)

In [ ]:
plot_path = utils.create_path(utils.PICUTRE_DIR / "buoy")
plot_trajectories([buoy_ds_sliced, pfile_path], [buoy_id, "Simulated path"], legend=True, titlestr=f"ERDDAP Comparison from {np.datetime64(start, 's')} to {np.datetime64(end, 's')}",savefile=plot_path / f"buoy_comp_erddap_forecast_{buoy_id}.png")